In [3]:
# Download Indian States Shapefile
import subprocess
from pathlib import Path


def download_file(url, output_dir="artifacts"):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    output_path = f'{output_dir}/{url.split("/")[-1]}'
    try:
        cmd = f"wget {url} --no-check-certificate -O {output_path}"
        subprocess.Popen(cmd, shell=True).wait()
    except Exception as e:
        print(e)


url = "https://indiaviz.org/resources/india_states.geojson"

download_file(url)

--2024-07-09 22:41:04--  https://indiaviz.org/resources/india_states.geojson
Resolving indiaviz.org (indiaviz.org)... 52.11.204.148
Connecting to indiaviz.org (indiaviz.org)|52.11.204.148|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 1725271 (1.6M) [application/octet-stream]
Saving to: ‘artifacts/india_states.geojson’

     0K .......... .......... .......... .......... ..........  2%  248K 7s
    50K .......... .......... .......... .......... ..........  5%  255K 6s
   100K .......... .......... .......... .......... ..........  8% 6.00M 4s
   150K .......... .......... .......... .......... .......... 11% 9.23M 3s
   200K .......... .......... .......... .......... .......... 14%  262K 3s
   250K .......... .......... .......... .......... .......... 17% 6.09M 3s
   300K .......... .......... .......... .......... .......... 20% 9.24M 2s
   350K .......... .......... .......... .......... .......... 23% 6.99M 2s
   400K .

In [25]:
# Dissolve the shapefile
import geopandas as gpd

states = gpd.read_file("artifacts/india_states.geojson")
india = states.copy()
india["dissolve_id"] = 1
india = india.dissolve(by="dissolve_id").reset_index()

# india.to_file('artifacts/india.geojson', driver='GeoJSON')

In [29]:
states.head()

,ST_NM,ST_ID,ID,geometry
0,Jammu & Kashmir,IN-JK,JK,"MULTIPOLYGON (((75.88864 32.57603, 75.84635 32..."
1,Gujarat,IN-GJ,GJ,"MULTIPOLYGON (((72.67653 21.1557, 72.70319 21...."
2,Gujarat,IN-GJ,GJ,"MULTIPOLYGON (((72.35319 21.61347, 72.36291 21..."
3,Gujarat,IN-GJ,GJ,"MULTIPOLYGON (((72.72264 21.66597, 72.74597 21..."
4,Gujarat,IN-GJ,GJ,"MULTIPOLYGON (((72.27431 21.74958, 72.28264 21..."


In [33]:
for state in states.ID.unique():
    # Get State Name
    state_name = states[states.ID == state].ST_NM.values[0]
    # Strip Special Characters and Spaces and Replace with Underscores
    state_name = state_name.replace(" ", "_").replace("&", "and")
    # Dissolve the shapefile and save as GeoJSON
    states[states.ID == state].dissolve(by="ID").reset_index().to_file(
        f"artifacts/{state_name}.geojson", driver="GeoJSON"
    )

/var/folders/fb/rn160t056lv_0jmjgq_f94680000gn/T/ipykernel_27845/3262992944.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  india.centroid.y.values


array([23.3788772])

In [19]:
import geemap.foliumap as geemap
import warnings

warnings.filterwarnings("ignore")
m = geemap.Map()
m.add_geojson("artifacts/india.geojson")
# Set Bounds to India
m.set_center(lat=india.centroid.y.values[0], lon=india.centroid.x.values[0], zoom=4)
m

In [20]:
import streamlit as st

help(st.sidebar)

Help on DeltaGenerator in module streamlit.delta_generator:

wrapper(*args: 'Any', **kwargs: 'Any') -> 'NoReturn'



In [21]:
help(m.add_gdf)

Help on method add_gdf in module geemap.foliumap:

add_gdf(gdf, layer_name='Untitled', zoom_to_layer=True, info_mode='on_hover', fields=None, **kwargs) method of geemap.foliumap.Map instance
    Adds a GeoPandas GeoDataFrame to the map.
    
    Args:
        gdf (GeoDataFrame): A GeoPandas GeoDataFrame.
        layer_name (str, optional): The layer name to be used. Defaults to "Untitled".
        zoom_to_layer (bool, optional): Whether to zoom to the layer.
        info_mode (str, optional): Displays the attributes by either on_hover or on_click.
            Any value other than "on_hover" or "on_click" will be treated as None. Defaults to "on_hover".
        fields (list, optional): The fields to be displayed in the popup. Defaults to None.



In [22]:
help(m.set_center)

Help on method set_center in module geemap.foliumap:

set_center(lon, lat, zoom=10) method of geemap.foliumap.Map instance
    Centers the map view at a given coordinates with the given zoom level.
    
    Args:
        lon (float): The longitude of the center, in degrees.
        lat (float): The latitude of the center, in degrees.
        zoom (int, optional): The zoom level, from 1 to 24. Defaults to 10.



In [24]:
m.st_fit_bounds()

2024-07-10 10:07:10.694 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
